# Session 2: The Ball & Stick Neuron

In [ ]:
# Import modules
from neuron import h, gui
import pylab as plt

## About this tutorial

In this tutorial you are going to use the [NEURON SIMULATOR](https://neuron.yale.edu). This is a tool to simulate biological neurons and networks of neurons with detailed morphologies. In this exercise we will create a simple cell with just a single dendrite and soma and investigate some effects of tweaking model parameters. This will set you up to set up more complex experiments with more interesting morphologies. This simple cell we call the *Ball & Stick Cell* representing the soma and dendrite, respectively.

Let's first import the right modules. For the neuron simulator to work in python we need to import its **h** module which contains all functionality for constructing and simulating the neurons. The **gui** module is also required, but for this exercise there is no need to understand its functionality. 

## Create sections
A neuron consists of multiple compartement with specific biophysical properties. in NEURON compartments of a cell are modelled as $Sections$. Each $Section$ has a number of $Segments$ ($nseg$). To create a section we simple call $h.Section$ and give it a name (the argument of the function).

In [ ]:
soma = h.Section(name='soma') # create the cell soma
# also create a dendrite
dend = # ------ your code here ------ #

In [ ]:
# h.psection prints information about the section 
h.psection(sec=soma) # print section
# h.topology prints the topology of the whole of sections that you created
h.topology()         # print topology

The soma and the dendrite both have 1 segment. The topology should show that they are unconnected and each section has 2 ends: 0 and 1. 

Now lets connect them!

## Topology
We connect the dendrite to the soma. The location of the connection on the $Section$ (so the $Segment$ of the $Section$ where the connection is made) runs from $0$ to $1$. In this case both the dendrite and the soma have 1 $Segment$ so we simply connect the dendrite to the "1" end of the soma. Connect and investigate the topology of the system.

In [ ]:
dend.connect(soma(1)) # connect dendrite at the '1' end of the soma

Now we have connected the "0" end of the dendrite to the "1" end of the soma.

Next we can define the spatial properties of the $Sections$ using the cells geometry.

## Geometry
The sections in NEURON have a cylinder shape. The surface area of a cylinder is $2\pi rh$. To get a ball shaped soma we need to set the length equal to it's diameter. We want a soma with a surface area of 500 microns squared.

### Exercise 1: Calculate the diameter and length of the soma to get a surface area of 500 microns squared
The diameter and the length should be the same for the soma to get a approximately ball shaped section.

In [ ]:
soma.L = soma.diam = # ------ your code here ------ #

The dendrite has an elongated shape, the length is much larger than the diameter.

In [ ]:
dend.L = 200  # microns
dend.diam = 1 # microns

The $Sections$ we made are not actually soma and dendrites, we just named and shaped them that way. To really simulate the dynamics of a soma and a dendrite we need to define the biophysical mechanisms of the $Sections$.

## Biophysics
In NEURON we can insert membrane properties, like leak currents and conductances. Here, we will set up the soma membrane with Hodgin-Huxley dynamics. The dendrite will have a simple passive membrane with a single conductance. We first set the resistance and membrane capacitance of both the soma and the dendrite using $h.allsec()$.

In [ ]:
for sec in h.allsec():
    sec.Ra = 100   # Axial resistance in Ohm*cm
    sec.cm = 1     # Membrane capacitance in mF/cm^2

Then we insert an active Hodgin-Huxley current for all the $Segments$ in the soma (if you didn't change the number of $Segments$ it should be 1) using $.insert()$

In [ ]:
soma.insert('hh') # insert active Hodgin-Huxley current

# set the parameters of the active membrane
for seg in soma:
    seg.hh.gnabar = 0.12   # Sodium conductance in S/cm^2
    seg.hh.gkbar  = 0.036  # Potassium conductance in S/cm^2
    seg.hh.gl     = 0.0003 # Leak conductance in S/cm^2
    seg.hh.el     = -54.3  # Reversal potential in mV

Finally we need to set up the biophysics of the dendrite with a passive current.

In [ ]:
# insert a passive membrane ('pas')
# ------ your code here ------ #

# set the passive conductance (S/cm^2) 'g = 0.001' and the leak reversal potential (mV) 'e = -65'.
# ------ your code here ------ #

Now the topology and biophysics of the model neuron is ready we can simulate and visualize the dynamics of the model.

## Simulation
To get any interesting dynamics we will inject a stimulus using the $h.IClamp()$ function. We inject it at the '1' end of the dendrite. Furthermore, we start it at 5 ms into the simulation, give it a duration of 1 ms and an amplitude of 0.1 nA.

In [ ]:
stim = h.IClamp(dend(1))
stim.delay = 5
stim.dur   = 1
stim.amp   = 0.1

To simulate we need to create recording vectors for the membrane potential and the time. These are used to store the recorded variables.

In [ ]:
v_vec = h.Vector()   # Membrane potential
t_vec = h.Vector()   # Time vector
v_vec.record(soma(0.5)._ref_v) # record from the middle of the soma
t_vec.record(h._ref_t)         # record time

Vector[4]

now we are ready to simulate!

In [ ]:
h.tstop = 25.0  # simulation time
h.run()

0.0

...and visualize!

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(t_vec, v_vec)
plt.xlabel('Time (ms)')
plt.ylabel('Membrane potential (mV)')
plt.show()

A spike! It seems the dendritic input caused the membrane potential in the soma to jump. Now we went through the basics of NEURON we can experiment with more interesting cases.

### Exercise 2: Varying the amplitude of the dendritic stimulation
Change the amplitude and duration of the dendritic stimulation. What happens to the membrane potential of the soma? Can you explain the dynamics?

### Exercise 3: Dendritic depolarization
Try plotting the dendritic depolarization at the middel of the dendrite. Compare it to the soma depolarization. Can you explain the differences in terms of amplitude and timing?

In [ ]:
t_vec = h.Vector()       # Time vector
t_vec.record(h._ref_t)   # record time

# record from the middle of the soma
s_vec = h.Vector()             # Membrane potential
s_vec.record(soma(0.5)._ref_v) # record from the middle of the soma

# record from the middle of the dendrite
# ------ your code here ------ #

# plot the trajectories
plt.figure(figsize=(8,4))
# ------ your code here ------ #
plt.xlabel('Time (ms)')
plt.ylabel('Membrane potential (mV)')
plt.show()

### Exercise 4: Dendritic resolution
in reality the biological dendrite does not behave as one compartment. Synaptic input can arrive at different locations along the dendrite. Further, the length and shape of the dendrite has an effect on the current that will eventually reach the soma. To incorporate these effects in our Ball & Stick model we need to increase the number of 𝑆𝑒𝑔𝑚𝑒𝑛𝑡𝑠 in our dendrite 𝑆𝑒𝑐𝑡𝑖𝑜𝑛. Try to set the number of 𝑆𝑒𝑔𝑚𝑒𝑛𝑡𝑠 of the dendrite (𝑛𝑠𝑒𝑔) to 101 (odd value because we evaluate the at the middle of the dendrite). What happens to the membrane potential of the dendrite and the soma when applying stimulation at different 𝑆𝑒𝑔𝑚𝑒𝑛𝑡𝑠 along the dendrite? Try experimenting with different dendrite lengths, number of 𝑆𝑒𝑔𝑚𝑒𝑛𝑡𝑠. Can you think of a computation that could be performed by a single dendrite with the potential to be stimulated (e.g. by other neuron's synapses) at different sites along it's shaft?

In [ ]:
# set the number of Segments (nseg) of the dendrite to 101.
# ------ your code here ------ #
h.topology()

# apply stimulation at a location along the dendritic shaft
# ------ your code here ------ #

# plot the membrane potential in the middle of the dendrite and the soma
# ------ your code here ------ #